In [1]:
import sys
sys.path.insert(1, '/kaggle/input/du2project-code/project')

In [12]:
import torch
import torchvision
import torch.utils.data as data
import os
from os.path import join
import argparse
import logging
from tqdm import tqdm
from torchvision.utils import save_image
#user import
from data_generator.DataLoader_Pretrain_Alexnet import CACD
from model.GAN import IPCGANs
from utils.io import check_dir,Img_to_zero_center,Reverse_zero_center
from datetime import datetime

In [3]:
TIMESTAMP = "{0:%Y-%m-%d_%H-%M-%S}".format(datetime.now())

In [72]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
    
args = {
    'learning_rate' : 1e-4,
    'batch_size' : 256,
    'max_epoches' : 200,
    'val_interval' : 20000,
    'save_interval' : 20000, 
    'cuda_device' : '0',
    'checkpoint' : '/kaggle/working/checkpoint/pretrain_alexnet',
    'saved_model_folder' : '/kaggle/working/checkpoint/pretrain_alexnet/saved_parameters',
    'list_root' : '/kaggle/input/du2project-code/project/data/cacd2000-lists',
    'data_root' : '/kaggle/input/du2project-data/CACD2000',
    
}

args = dotdict(args)

In [48]:
os.mkdir(args.checkpoint)
os.mkdir(args.saved_model_folder)

In [50]:
from model.faceAlexnet import AgeClassify
model=AgeClassify()
optim=model.optim

In [61]:
class CACD(data.Dataset):
    def __init__(self, list_root="C:/Users/Frane/Desktop/DIPLOMSKI/peta/duboko_ucenje2/projekt/data/cacd2000-lists",
                 data_root="C:/Users/Frane/Desktop/DIPLOMSKI/peta/duboko_ucenje2/projekt/CACD2000",
                 split='train', transforms=None, label_transforms=None):
        
        """list_root = "C:/Users/Frane/Desktop/DIPLOMSKI/peta/duboko_ucenje2/projekt/data/cacd2000-lists"
        data_root = "C:/Users/Frane/Desktop/DIPLOMSKI/peta/duboko_ucenje2/projekt/CACD2000"""
        
        if split == "train":
            self.list_path=os.path.join(list_root,"train.txt")
        else:
            self.list_path = os.path.join(list_root, "test.txt")
        self.images_labels=[]#path
        self.transform=transforms
        self.label_transform=label_transforms
        with open(self.list_path,errors='replace') as fr:
            lines=fr.readlines()
            for line in lines:
                line.strip()
                item=line.split()
                image_label=[]
                image_label.append(os.path.join(data_root,item[0]))
                image_label.append(np.array(item[1],dtype=np.int32))
                self.images_labels.append(image_label)

    def __getitem__(self, idx):
        img_path,label=self.images_labels[idx]
        img=Image.open(img_path)

        if self.transform is not None:
            img=self.transform(img)
        if self.label_transform is None:
            label=torch.from_numpy(label)
        return img,label

    def __len__(self):
        return len(self.images_labels)

In [73]:
transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((227, 227)),
        torchvision.transforms.ToTensor(),
        Img_to_zero_center()
    ])
#step4: define train/test dataloader
train_dataset = CACD(
    list_root = args.list_root, 
    data_root= args.data_root, 
    split = "train",
    transforms=transforms, label_transforms=None
)
test_dataset = CACD(
    list_root = args.list_root, 
    data_root= args.data_root, 
    split = "test",
    transforms=transforms, label_transforms=None
)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=args.batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=args.batch_size,
    shuffle=False
)

In [74]:
for epoch in range(args.max_epoches):
    pbar = tqdm(enumerate(train_loader),total = len(train_loader))
    for train_idx, (img,label) in pbar:
        img=img.cuda()
        label=label.type(torch.LongTensor)
        label=label.cuda()

        #train
        optim.zero_grad()
        model.train(img,label)
        loss=model.loss
        loss.backward()
        optim.step()
        format_str = ('step %d/%d, cls_loss = %.3f')
        pbar.set_postfix({'cls_loss': '%.3f' %(loss)})


        # save the parameters at the end of each save interval
        if train_idx*args.batch_size % args.save_interval == 0 and train_idx != 0:
            model.save_model(dir=args.saved_model_folder,
                             filename='epoch_%d_iter_%d.pth'%(epoch, train_idx))
            print('checkpoint has been created!')

        #val step

        if train_idx % args.val_interval == 0 and train_idx != 0:
            train_correct=0
            train_total=0
            with torch.no_grad():
                for val_img,val_label in tqdm(test_loader):
                    val_img=val_img.cuda()
                    val_label=val_label.cuda()
                    output=model.val(val_img)
                    train_correct += (output == val_label).sum()
                    train_total += val_img.size()[0]

            print('validate has been finished!')
            format_str = ('val_acc = %.3f')
            print(format_str % (train_correct.cpu().numpy()/train_total))

  6%|▌         | 32/574 [01:15<21:15,  2.35s/it, cls_loss=1.475]


KeyboardInterrupt: 